## Using Presto to Generate Reports

To generate reports for those interested in application usage, we can use Presto via the PyHive connector to load our Hive tables into Pandas dataframes, transform the dataframes to answer our business questions, and then publish the reports as JSON files.

Before we start, let's define two business questions that we would like to answer:

1. What are all the counts per event type?
2. What are all the parameters that were given for the `user` parameter?

First, let's install the PyHive library.

In [1]:
!pip install pyhive

The directory '/w205/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/w205/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |################################| 51kB 931kB/s ta 0:00:01
    100% |################################| 849kB 2.2MB/s ta 0:00:011
    100% |################################| 256kB 4.4MB/s ta 0:00:01
  Running setup.py install for future ... done
  Running setup.py install for pyhive ... done
You are using pip version 8.1.1, however version 23.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
!python3 -m pip install git+https://github.com/dropbox/PyHive.git

The directory '/w205/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/w205/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
  Cloning https://github.com/dropbox/PyHive.git to /tmp/pip-5d4aqpmx-build
You are using pip version 8.1.1, however version 23.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Next, let's use PyHive to connect to Presto in code using the port that we exposed in our Docker Compose file. Once we connect, we can run a simple query to see all the tables that are created in Hive.

In [10]:
from pyhive import presto
import pandas as pd

presto_conn = presto.connect(
    host='presto',
    port=8080 # Exposed Presto port (see docker compose file)
)

pd.read_sql_query("SHOW TABLES", presto_conn)

,Table
0,all_events
1,event_parameters


Now let's run a query to get all of the data from the `event_parameters` table and load it into a Pandas dataframe.

In [11]:
# https://stackoverflow.com/questions/55988436/how-to-convert-a-presto-query-output-to-a-python-data-frame
event_parameters = pd.read_sql_query("SELECT * from event_parameters", presto_conn)
event_parameters.head()

,raw_event,timestamp,accept,host,user_agent,event_id,parameter_name,parameter_value
0,"{""event_id"": ""d064a912-1b64-4e2b-bfe8-97175b01...",2023-08-18 03:54:40.09,None,None,None,d064a912-1b64-4e2b-bfe8-97175b01aa54,sword_type,Final Words
1,"{""event_id"": ""d064a912-1b64-4e2b-bfe8-97175b01...",2023-08-18 03:54:40.09,None,None,None,d064a912-1b64-4e2b-bfe8-97175b01aa54,user,don
2,"{""event_id"": ""a7c86f52-a2e8-4214-9cbb-0e16e3fc...",2023-08-18 03:54:40.092,None,None,None,a7c86f52-a2e8-4214-9cbb-0e16e3fce72e,sword_type,Final Words
3,"{""event_id"": ""a7c86f52-a2e8-4214-9cbb-0e16e3fc...",2023-08-18 03:54:40.092,None,None,None,a7c86f52-a2e8-4214-9cbb-0e16e3fce72e,user,don
4,"{""event_id"": ""2e91877f-26e7-446d-ac79-3333dccf...",2023-08-18 03:54:40.094,None,None,None,2e91877f-26e7-446d-ac79-3333dccffc9c,sword_type,Final Words


Let's do the same thing for the data in the `all_events` table.

In [12]:
all_events = pd.read_sql_query("SELECT * from all_events", presto_conn)
all_events.head()

,raw_event,timestamp,accept,host,user_agent,event_id,event_type
0,"{""event_id"": ""73927aa9-6668-4915-8550-a25677b3...",2023-08-18 03:52:10.242,*/*,user2.att.com,ApacheBench/2.3,73927aa9-6668-4915-8550-a25677b39afd,join_guild
1,"{""event_id"": ""00b0f1ae-1b9e-4412-956d-89d1f53a...",2023-08-18 03:52:10.244,*/*,user2.att.com,ApacheBench/2.3,00b0f1ae-1b9e-4412-956d-89d1f53ac6ee,join_guild
2,"{""event_id"": ""e87e9ec6-1439-44b2-8f4a-89f5aff3...",2023-08-18 03:52:10.246,*/*,user2.att.com,ApacheBench/2.3,e87e9ec6-1439-44b2-8f4a-89f5aff322b1,join_guild
3,"{""event_id"": ""d1adf5b4-1177-45d2-85f7-31574eaf...",2023-08-18 03:52:10.248,*/*,user2.att.com,ApacheBench/2.3,d1adf5b4-1177-45d2-85f7-31574eaf97d9,join_guild
4,"{""event_id"": ""50e2dc3c-75ec-4b9c-9d2d-e21a3b60...",2023-08-18 03:52:10.249,*/*,user2.att.com,ApacheBench/2.3,50e2dc3c-75ec-4b9c-9d2d-e21a3b601f8e,join_guild


Now let's try to answer business question #1 - What are all the counts per event type? We can do this using a simple groupby statement on our `all_events` dataframe and then writing the output to a JSON file titled `event_type_count.json`.

In [14]:
event_type_count = all_events.groupby('event_type').size()
event_type_count.to_json("event_type_count.json", orient='columns')

Note that `event_type_count.json` should exist in your directory after running the code above.


In [15]:
all_events = pd.read_sql_query("select event_type, count(event_type) as event_count from all_events group by event_type", presto_conn)
all_events.head()

,event_type,event_count
0,join_guild,5143
1,get_credit,5870
2,purchase_sword,5499
3,leave_guild,6087




Now let's answer business question #2 - What are all the parameters that were given for the `user` parameter? We can do this by running a slightly more complex query on our dataframe and again writing the output to a JSON file.

In [16]:
# Question: What are all the parameters that were given for the `user` parameter?
user_parameter_count = event_parameters.where(event_parameters['parameter_name'] == 'user').groupby('parameter_value').size()
user_parameter_count.to_json("user_parameter_count.json", orient='columns')

events_by_user = pd.read_sql_query(" select     un.parameter_value as user,     et.event_type as event,    count(un.parameter_value) as user_event_count from     all_events et join      event_parameters un on      et.event_id = un.event_id and     un.parameter_name = 'user' where      un.parameter_name = 'user'\
and     et.event_id = un.event_id group by     un.parameter_value     ,et.event_type order by     et.event_type,     un.parameter_value ", presto_conn)

events_by_user.head(50)


,user,event,user_event_count
0,aastha,get_credit,1026
1,ben,get_credit,1233
2,don,get_credit,1051
3,lise,get_credit,1348
4,theresa,get_credit,1317
5,aastha,join_guild,1092
6,ben,join_guild,866
7,don,join_guild,1260
8,lise,join_guild,1120
9,theresa,join_guild,908


Now that we have answered the business questions and generated JSON reports with the answers, we can close our Presto connection.

In [24]:
presto_conn.close()